# Certified Finetuning of a Classifier on the OCT-MNIST Dataset

In [2]:
%load_ext autoreload
%autoreload 2
import os
import torch
import tqdm
import abstract_gradient_training as agt
from abstract_gradient_training import AGTConfig
from abstract_gradient_training import certified_training_utils as ct_utils
from models.deepmind import DeepMindSmall 
from datasets import oct_mnist

## Pre-train the model

Exclude class 2 (Drusen) from the pretraining.

In [3]:
# set up pre-training
torch.manual_seed(1)
device = torch.device("cuda:0")
pretrain_batchsize = 100
pretrain_n_epochs = 20
pretrain_learning_rate = 0.001

In [4]:
# define model, dataset and optimizer
model = DeepMindSmall(1, 1)
dl_pretrain, _ = oct_mnist.get_dataloaders(pretrain_batchsize, exclude_classes=[2], balanced=True)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_learning_rate)
model = model.to(device)

In [5]:
if os.path.exists(".models/medmnist.ckpt"):
    model.load_state_dict(torch.load(".models/medmnist.ckpt"))
else:  # pre-train the model
    progress_bar = tqdm.trange(pretrain_n_epochs, desc="Epoch", )
    for epoch in progress_bar:
        for i, (x, u) in enumerate(dl_pretrain):
            # Forward pass
            u, x = u.to(device), x.to(device)
            output = model(x)
            loss = criterion(output.squeeze().float(), u.squeeze().float())
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                progress_bar.set_postfix(loss=loss.item())
    # save the model
    with open(".models/medmnist.ckpt", "wb") as file:
        torch.save(model.state_dict(), file)

Epoch: 100%|██████████| 20/20 [00:40<00:00,  2.00s/it, loss=0.0199] 


## Fine-tune the model on the private Drusen data

In [6]:
# set up fine-tuning parameters
batchsize = 3000
config = AGTConfig(
    fragsize=1000,
    learning_rate=0.03,
    n_epochs=1,
    k_private=10,
    clip_gamma=2.0,
    dp_sgd_sigma=0.01,
    forward_bound="interval",
    device="cuda:0",
    backward_bound="interval",
    loss="binary_cross_entropy",
    log_level="DEBUG",
    lr_decay=0.3,
    lr_min=0.001,
    early_stopping=False,
)

In [7]:
# get dataloaders, train dataloader is a mix of drusen and the "healthy" class
dl_train, _ = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[0, 1], balanced=True)
_, dl_test_drusen = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[0, 1, 3])
_, dl_test_other = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[2])
_, dl_test_all = oct_mnist.get_dataloaders(batchsize, 1000)

In [8]:
# evaluate the pre-trained model
param_n, param_l, param_u = ct_utils.get_parameters(model)
drusen_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_drusen)), model, ct_utils.propagate_conv_layers)
other_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_other)), model, ct_utils.propagate_conv_layers)
all_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers)

print("=========== Pre-trained model accuracy ===========")
print(f"Class 2 (Drusen) : nominal = {drusen_acc[1]:.2g}")
print(f"Classes 0, 1, 3  : nominal = {other_acc[1]:.2g}")
print(f"All Classes      : nominal = {all_acc[1]:.2g}")

=========== Pre-trained model accuracy ===========
Class 2 (Drusen) : nominal = 0.46
Classes 0, 1, 3  : nominal = 0.96
All Classes      : nominal = 0.84


In [9]:
# fine-tune the model using abstract gradient training (keeping the convolutional layers fixed)
param_l, param_n, param_u = agt.privacy_certified_training(
    model, config, dl_train, dl_test_drusen, transform=ct_utils.propagate_conv_layers
)

[AGT] [INFO    ] [17:30:15] =================== Starting Privacy Certified Training ===================
[AGT] [DEBUG   ] [17:30:15] 	Privacy parameters: k_private=10, clip_gamma=2.0, dp_sgd_sigma=0.01
[AGT] [DEBUG   ] [17:30:15] 	Bounding methods: forward=interval, backward=interval
[AGT] [INFO    ] [17:30:15] Starting epoch 1
[AGT] [DEBUG   ] [17:30:15] Initialising dataloader batchsize to 3000
[AGT] [INFO    ] [17:30:15] Training batch 0: Network eval bounds=(0.46, 0.46, 0.46), W0 Bound=0.0 
[AGT] [INFO    ] [17:30:16] Training batch 1: Network eval bounds=(0.52, 0.52, 0.53), W0 Bound=0.23 
[AGT] [INFO    ] [17:30:16] Training batch 2: Network eval bounds=(0.54, 0.58, 0.6 ), W0 Bound=0.407 
[AGT] [INFO    ] [17:30:17] Training batch 3: Network eval bounds=(0.58, 0.6 , 0.64), W0 Bound=0.552 
[AGT] [INFO    ] [17:30:17] Training batch 4: Network eval bounds=(0.59, 0.63, 0.69), W0 Bound=0.674 
[AGT] [DEBUG   ] [17:30:18] Skipping batch 5 in epoch 1 (expected size 3000, got 508)
[AGT] [I

In [10]:
# evaluate the fine-tuned model
drusen_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_drusen)), model, ct_utils.propagate_conv_layers)
other_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_other)), model, ct_utils.propagate_conv_layers)
all_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers)

print("=========== Fine-tuned model accuracy + bounds ===========")
print(f"Class 2 (Drusen) : nominal = {drusen_acc[1]:.2g}, certified bound = {drusen_acc[0]:.2g}")
print(f"Classes 0, 1, 3  : nominal = {other_acc[1]:.2g}, certified bound = {other_acc[0]:.2g}")
print(f"All Classes      : nominal = {all_acc[1]:.2g}, certified bound = {all_acc[0]:.2g}")

=========== Fine-tuned model accuracy + bounds ===========
Class 2 (Drusen) : nominal = 0.66, certified bound = 0.6
Classes 0, 1, 3  : nominal = 0.93, certified bound = 0.92
All Classes      : nominal = 0.87, certified bound = 0.84
